In [14]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score

In [15]:
datafile1 = pd.ExcelFile('data/SDE_DATA_1998.xlsx')

In [16]:
data1998 = datafile1.parse()

In [17]:
# show how the data looks like
data1998.loc[:3, ]

,Country,Region,Sub Region,Road Maintenance,Road Network,Road maintenance per network,Net Energy Imports,Trade as a % of GDP,Access to Electricity,Quality of Electricity Supply,...,FDI inflows,RnD Expenditure as a % of GDP,Payments for IP use,Charges for IP use,Charges for IP use as % GDP,Hi Tech Exports as % of manufactured exports,GDP Growth,GDP Growth per capita,Population,GDP
0,Indicator,Indicator,Indicator,Euro,km,Euro/km,%,%,%,Rank,...,%,%,$ (USD),$ (USD),%,%,%,%,#,#
1,Afghanistan,South Asia,South Asia,NaN,42150,0,NaN,NaN,81.1357,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18034130,NaN
2,Albania *,Europe and Central Asia,Balkans,2770395,18000,153.911,16.6227,45.4813,100,NaN,...,NaN,NaN,NaN,NaN,0,0.339874,9.00885,9.69705,3148281,2.70712e+09
3,Algeria,Middle East and North Africa,North Africa,NaN,113655,0,-410.452,45.0945,34.6157,NaN,...,4.88931,NaN,NaN,NaN,0,1.17094,5.1,3.54391,29887717,4.81877e+10


In [18]:
# prepare the data

label198 = 28  # GDP growth per capita
features1998 = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 29]

imputer = Imputer(axis=1)
X_dirty = data1998.values[1:, features1998].astype(float)
X = imputer.fit_transform(X_dirty)
y_dirty = data1998.values[1:, label198].astype(float)
y = imputer.fit_transform(y_dirty.reshape(1, -1)).ravel()

In [23]:
# do PCA with a linear regression

linear = linear_model.LinearRegression()
pca = decomposition.PCA()
pipe = Pipeline(steps=[('pca', pca), ('linear', linear)])
score = cross_val_score(pipe, X, y).mean()
print("Score = %.2f" % score)

Score after imputation of the missing values = -238986468.63


In [20]:
# do linear regression only 
estimator = linear_model.LinearRegression()
score = cross_val_score(estimator, X, y).mean()
print("Score = %.2f" % score)

Score after imputation of the missing values = -238986468.63


In [21]:
estimator = RandomForestRegressor()
score = cross_val_score(estimator, X, y).mean()
print("Score = %.2f" % score)

Score after imputation of the missing values = -0.34


In [24]:
cross_val_score(estimator, X, y)

array([-1.06641055,  0.01798894, -0.56472055])